In [21]:
# All Features Dimension = 5183

# ECFP(1024) + Pubchem FP(881) + Descriptors(11)=1916

# Descriptors calculated by mordred (1826)

# Interaction Fingerprint(1440) + Docking Score (1)=1441

5183-1916-1826-1441

0

ML

In [1]:
from featurization import compute_fps, compute_physchem, compute_fingerprint_features, compute_descriptor_features
import numpy as np
from typing import List
import pandas as pd
import pickle
%reload_ext autoreload
%autoreload 2
from new_unit import Data, get_config, cross_validate, f1_acc
from models import RF, SVM, GBM,  GCN, MPNN, CNN, Transformer
from rdkit.Chem import Descriptors as Des
from rdkit import Chem

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, roc_curve,precision_recall_curve, auc
from math import sqrt
from unit import compute_metrics
np.random.seed(42)

In [2]:
Moldel_Type = "ML"
Model = "GBM"
FingerPrint = "2FP"
Des_IF = "11Des_1441IF"

file_name = f"./trained_models/{Moldel_Type}/{Model}_{FingerPrint}_{Des_IF}_forAUC.csv"
model_name = f"./trained_models/{Moldel_Type}/{Model}_{FingerPrint}_{Des_IF}.pkl"

# hyperparameters = get_config(f"./configures/optimized/train_split/{Model}_ECFP.yml")
# if "epochs" in hyperparameters:
#     hyperparameters.pop("epochs")
# f = GBM(**hyperparameters)

with open(model_name, 'rb') as handle:
    f = pickle.load(handle)

In [3]:
with open(file_name,"w") as p:
    p.write("auc_score, pr_auc, AC, f1, SN, PR, SP, CCR, MCC\n")

In [4]:
df = pd.read_csv("./data/IF_Data/Train_Val_IF.csv")
df['r_i_docking_score'] = df['r_i_docking_score'].apply(lambda x: 0 if x > 0 else x)
df["MW"]=df["smiles"].apply(lambda x : Des.ExactMolWt(Chem.MolFromSmiles(x)))


df_train = df[df["split"]=="train"].reset_index()
df_val = df[df["split"]=="test"].reset_index()

train_mask = (df_train['MW'] >= 300) & (df_train['MW'] <= 500)
val_mask = (df_val['MW'] >= 300) & (df_val['MW'] <= 500)

train_Ro5_idx = df_train[train_mask].index
val_Ro5_idx = df_val[val_mask].index


In [5]:
df_Test_1 = pd.read_csv("./data/IF_Data/Test_1_IF.csv")
df_Test_1['r_i_docking_score'] = df_Test_1['r_i_docking_score'].apply(lambda x: 0 if x > 0 else x)
df_Test_1["MW"]=df_Test_1["smiles"].apply(lambda x : Des.ExactMolWt(Chem.MolFromSmiles(x)))

# df_Test_1 = df_Test_1.reset_index()

Test_1_mask = (df_Test_1['MW'] >= 300) & (df_Test_1['MW'] <= 500)
Test_1_Ro5_idx = df_Test_1[Test_1_mask].index

In [6]:
Test_1_Ro5_idx.shape

(177,)

In [7]:
df_Test_2 = pd.read_csv("./data/IF_Data/Test_2_IF.csv")
df_Test_2['r_i_docking_score'] = df_Test_2['r_i_docking_score'].apply(lambda x: 0 if x > 0 else x)
df_Test_2["MW"]=df_Test_2["smiles"].apply(lambda x : Des.ExactMolWt(Chem.MolFromSmiles(x)))

# df_Test_2 = df_Test_2.reset_index()

Test_2_mask = (df_Test_2['MW'] >= 300) & (df_Test_2['MW'] <= 500)
Test_2_Ro5_idx = df_Test_2[Test_2_mask].index

In [8]:
Test_2_Ro5_idx.shape

(379,)

In [9]:
train_all_rep = "./data/IF_Data/Train_All_Features.pkl"
with open(train_all_rep, 'rb') as handle:
    train_all_rep = pickle.load(handle)[train_Ro5_idx]

val_all_rep = "./data/IF_Data/Val_All_Features.pkl"
with open(val_all_rep, 'rb') as handle:
    val_all_rep = pickle.load(handle)[val_Ro5_idx]

Test_1_all_rep = "./data/IF_Data/Test_1_All_Features.pkl"
with open(Test_1_all_rep, 'rb') as handle:
    Test_1_all_rep = pickle.load(handle)[Test_1_Ro5_idx]

Test_2_all_rep = "./data/IF_Data/Test_2_All_Features.pkl"
with open(Test_2_all_rep, 'rb') as handle:
    Test_2_all_rep = pickle.load(handle)[Test_2_Ro5_idx]

In [10]:
Test_1_all_rep.shape

(177, 5183)

In [11]:
Test_2_all_rep.shape

(379, 5183)

In [12]:
# Train
# f.train(train_all_rep[:,:1916], np.array(df_train[train_mask]["y"]))
# with open(model_name, 'wb') as handle:
#     pickle.dump(f, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
### Validation

# y_hat = f.predict(val_all_rep[:,:1916])
# y_hat_proba = f.predict_prob(val_all_rep[:,:1916])[:, 1]

y_hat = f.predict(np.hstack((val_all_rep[:, :1916], val_all_rep[:, 3742:])))
y_hat_proba = f.predict_prob(np.hstack((val_all_rep[:, :1916], val_all_rep[:, 3742:])))[:, 1]

auc_score, pr_auc, AC, f1, SN, PR, SP, CCR, MCC = compute_metrics(df_val[val_mask]["y"], y_hat_proba, file_name)

f1, acc = f1_acc(np.array(df_val[val_mask]["y"]), y_hat)
fp_val, tpr_val, _ = roc_curve(np.array(df_val[val_mask]["y"]), y_hat_proba)
roc_auc_val = auc(fp_val, tpr_val)
print(f1)

Binary classification performace metrics:
[[583  87]
 [107 340]]
TP, FN, TN, FP
340, 107, 583, 87
0.7780320366132725


In [14]:
### Test 1

# y_hat = f.predict(Test_1_all_rep[:,:1916])
# y_hat_proba = f.predict_prob(Test_1_all_rep[:,:1916])[:, 1]
y_hat = f.predict(np.hstack((Test_1_all_rep[:, :1916], Test_1_all_rep[:, 3742:])))
y_hat_proba = f.predict_prob(np.hstack((Test_1_all_rep[:, :1916], Test_1_all_rep[:, 3742:])))[:, 1]

auc_score, pr_auc, AC, f1, SN, PR, SP, CCR, MCC = compute_metrics(df_Test_1[Test_1_mask]["y"], y_hat_proba, file_name)
f1, acc = f1_acc(np.array(df_Test_1[Test_1_mask]["y"]), y_hat)
fp_t1, tpr_t1, _= roc_curve(np.array(df_Test_1[Test_1_mask]["y"]), y_hat_proba)
roc_auc_t1 = auc(fp_t1, tpr_t1)
print(f1)

Binary classification performace metrics:
[[109  20]
 [ 20  28]]
TP, FN, TN, FP
28, 20, 109, 20
0.5833333333333334


In [15]:
df_Test_1[Test_1_mask]["y"].value_counts()

0    129
1     48
Name: y, dtype: int64

In [16]:
### Test 2

# y_hat = f.predict(Test_2_all_rep[:,:1916])
# y_hat_proba = f.predict_prob(Test_2_all_rep[:,:1916])[:, 1]
y_hat = f.predict(np.hstack((Test_2_all_rep[:, :1916], Test_2_all_rep[:, 3742:])))
y_hat_proba = f.predict_prob(np.hstack((Test_2_all_rep[:, :1916], Test_2_all_rep[:, 3742:])))[:, 1]

auc_score, pr_auc, AC, f1, SN, PR, SP, CCR, MCC = compute_metrics(df_Test_2[Test_2_mask]["y"], y_hat_proba, file_name)
f1, acc = f1_acc(np.array(df_Test_2[Test_2_mask]["y"]), y_hat)
fp_t2, tpr_t2,_ = roc_curve(np.array(df_Test_2[Test_2_mask]["y"]), y_hat_proba)
roc_auc_t2 = auc(fp_t2, tpr_t2)
print(f1)

Binary classification performace metrics:
[[236  39]
 [ 36  68]]
TP, FN, TN, FP
68, 36, 236, 39
0.6445497630331753


In [17]:
df_Test_2[Test_2_mask]["y"].value_counts()

0    275
1    104
Name: y, dtype: int64

In [ ]:
import numpy as np
# import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler


import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

import matplotlib.pyplot as plt
plt.rc('font',family='Times New Roman')



# # 计算各模型的ROC曲线及AUC值
# fp_val, tpr_val, _ = roc_curve(np.array(df_val[val_mask]["y"]), y_hat_val)
# roc_auc_val = auc(fp_val, tpr_val)

# fp_t1, tpr_t1, _= roc_curve(np.array(df_Test_1[Test_1_mask]["y"]), y_hat_t1)
# roc_auc_t1 = auc(fp_t1, tpr_t1)

# fp_t2, tpr_t2,_ = roc_curve(np.array(df_Test_2[Test_2_mask]["y"]), y_hat_t2)
# roc_auc_t2 = auc(fp_t2, tpr_t2)

# 绘制多个模型的ROC曲线
plt.figure(figsize=(12, 10), dpi=600)

plt.plot(fp_val, tpr_val, color='darkorange', lw=2, label=f'Validation (AUC = {roc_auc_val:.2f})')
plt.plot(fp_t1, tpr_t1, color='green', lw=2, label=f'Test 1 (AUC = {roc_auc_t1:.2f})')
plt.plot(fp_t2, tpr_t2, color='blue', lw=2, label=f'Test 2 (AUC = {roc_auc_t2:.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

# 修改刻度字体大小
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=20)
plt.ylabel('True Positive Rate', fontsize=20)
plt.title('ROC Curves for GBM on Validation, Test 1, and Test 2', fontsize=20)
plt.legend(loc="lower right", fontsize=20)

# 保存为矢量图（PDF格式）
plt.savefig("/home/ubuntu/FEAOF/figs/fig4B_2.pdf", format='pdf', bbox_inches='tight')
plt.show()
